In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL of the inventory page
base_url = "https://www.sbtjapan.com/used-cars/japan-inventory/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List to store all car data
all_cars = []

def scrape_page(url):
    """Scrape car listings from a single page."""
    print(f"Scraping: {url}")
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch page: {response.status_code}")
        return False

    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')

    for listing in listings:
        try:
            # Extract Stock ID
            stock_id = listing['id'].split('_')[-1]

            # Extract Make and Model from detail URL
            detail_link = listing.find('h2').find('a')['href']
            make, model = detail_link.split('/')[-3:-1]

            # Extract Year and Month
            year_month = listing.find('h2').text.strip().split()[0]
            year, month = year_month.split('/')

            # Extract Prices
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip() if len(prices) > 0 else "N/A"
            total_price = prices[1].text.strip() if len(prices) > 1 else "N/A"

            # Extract Specifications
            specs_table = listing.find('ul', class_='speclist_table')
            specs = specs_table.find_all('li') if specs_table else []
            mileage = specs[0].text.strip() if len(specs) > 0 else "N/A"
            engine = specs[1].text.strip() if len(specs) > 1 else "N/A"
            transmission = specs[2].text.strip() if len(specs) > 2 else "N/A"

            # Extract Image URL
            img_tag = listing.find('img')
            image_url = img_tag.get('data-original', "N/A") if img_tag else "N/A"

            # Store car data
            car_data = {
                'stock_id': stock_id,
                'make': make,
                'model': model,
                'year': year,
                'month': month,
                'fob_price': fob_price,
                'total_price': total_price,
                'mileage': mileage,
                'engine': engine,
                'transmission': transmission,
                'image_url': image_url
            }
            all_cars.append(car_data)

        except Exception as e:
            print(f"Error processing listing {stock_id}: {e}")

    # Check for next page
    next_page = soup.find('a', rel='next')
    if next_page:
        next_url = next_page['href']
        if not next_url.startswith('http'):
            next_url = "https://www.sbtjapan.com" + next_url
        time.sleep(1)  # Be polite to the server
        return next_url
    return False

# Start scraping from the base URL
current_url = base_url
while current_url:
    next_url = scrape_page(current_url)
    current_url = next_url

# Save data to CSV
df = pd.DataFrame(all_cars)
df.to_csv('sbtjapan_cars.csv', index=False)
print(f"Scraped {len(all_cars)} cars. Data saved to 'sbtjapan_cars.csv'.")

Scraping: https://www.sbtjapan.com/used-cars/japan-inventory/
Scraped 25 cars. Data saved to 'sbtjapan_cars.csv'.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL of the inventory page
base_url = "https://www.sbtjapan.com/used-cars/japan-inventory/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List to store all car data
all_cars = []

def scrape_page(url):
    """Scrape car listings from a single page and return the next page URL if available."""
    print(f"Scraping: {url}")
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch page: {response.status_code}")
        return False

    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')

    for listing in listings:
        try:
            # Extract Stock ID from the 'id' attribute
            stock_id = listing['id'].split('_')[-1]

            # Extract Make and Model from the detail URL
            detail_link = listing.find('h2').find('a')['href']
            url_parts = detail_link.split('/')
            make, model = url_parts[-3], url_parts[-2]  # e.g., 'toyota', 'harrier'

            # Extract Year and Month from <h2> text (e.g., "2018/5 TOYOTA HARRIER...")
            h2_text = listing.find('h2').text.strip()
            year_month = h2_text.split()[0]  # "2018/5"
            year, month = year_month.split('/')

            # Extract and clean Prices
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 0 else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"

            # Extract Specifications (mileage, engine, transmission)
            specs_table = listing.find('ul', class_='speclist_table')
            specs = specs_table.find_all('li') if specs_table else []
            mileage = specs[0].find('p').text.strip() if len(specs) > 0 else "N/A"  # e.g., "28,000km"
            engine = specs[1].find('p').text.strip() if len(specs) > 1 else "N/A"  # e.g., "2,690cc (2TR)"
            transmission = specs[2].find('p').text.strip() if len(specs) > 2 else "N/A"  # e.g., "AT"

            # Store data in a dictionary (excluding image_url)
            car_data = {
                'stock_id': stock_id,
                'make': make,
                'model': model,
                'year': year,
                'month': month,
                'fob_price': fob_price,
                'total_price': total_price,
                'mileage': mileage,
                'engine': engine,
                'transmission': transmission
            }
            all_cars.append(car_data)

        except Exception as e:
            print(f"Error processing listing {stock_id}: {e}")

    # Check for the next page
    next_page = soup.find('a', rel='next')
    if next_page:
        next_url = next_page['href']
        if not next_url.startswith('http'):
            next_url = "https://www.sbtjapan.com" + next_url
        time.sleep(1)  # Delay to avoid overwhelming the server
        return next_url
    return False

# Start scraping from the base URL and iterate through all pages
current_url = base_url
while current_url:
    next_url = scrape_page(current_url)
    current_url = next_url

# Define the column order for the CSV
columns = ['stock_id', 'make', 'model', 'year', 'month', 'fob_price', 'total_price', 'mileage', 'engine', 'transmission']

# Save data to CSV using pandas
df = pd.DataFrame(all_cars, columns=columns)
df.to_csv('sbtjapan_cars.csv', index=False)
print(f"Scraped {len(all_cars)} cars. Data saved to 'sbtjapan_cars.csv'.")

Scraping: https://www.sbtjapan.com/used-cars/japan-inventory/
Scraped 25 cars. Data saved to 'sbtjapan_cars.csv'.


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0'}

def scrape_list_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch list page: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            stock_id = listing['id'].split('_')[-1]
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    return car_data_list

# Main process
list_url = "https://www.sbtjapan.com/used-cars/japan-inventory/"  # Replace with your actual list page URL
car_list_data = scrape_list_page(list_url)

# Define columns (adjust based on what your previous scraper collected)
columns = ['stock_id', 'fob_price', 'total_price']

# Create DataFrame and save to CSV
df = pd.DataFrame(car_list_data)
df = df[columns]
df.to_csv('car_listings.csv', index=False)
print("Scraping complete. Data saved to 'car_listings.csv'.")

Scraping complete. Data saved to 'car_listings.csv'.


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {'User-Agent': 'Mozilla/5.0'}

# Scrape the list page for stock_id, prices, and detail URLs
def scrape_list_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch list page: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            # Extract Stock ID
            stock_id = listing['id'].split('_')[-1]
            
            # Extract Prices
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            
            # Extract Detail URL
            a_tag = listing.find('h2').find('a')
            detail_url = a_tag['href']
            if not detail_url.startswith('http'):
                detail_url = "https://www.sbtjapan.com" + detail_url
            
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price,
                'detail_url': detail_url
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    return car_data_list

# Scrape the detail page for additional fields
def scrape_detail_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch detail page: {response.status_code}")
        return {}
    
    soup = BeautifulSoup(response.text, 'html.parser')
    car_data = {}
    
    # Extract Make and Model from <h3>
    h3 = soup.find('h3')
    if h3:
        full_name = h3.text.split(" - ")[0].strip()
        parts = full_name.split(" ", 1)
        car_data['make'] = parts[0]
        car_data['model'] = parts[1] if len(parts) > 1 else ""
    
    # Extract other fields from the table
    table = soup.find('table', class_='tabA')
    if table:
        for tr in table.find_all('tr'):
            ths = tr.find_all('th')
            tds = tr.find_all('td')
            for i in range(len(ths)):
                key = ths[i].text.strip().lower().replace(' ', '_')
                value = tds[i].text.strip() if i < len(tds) else "N/A"
                car_data[key] = value
    
    return car_data

# Main process
list_url = "https://www.sbtjapan.com/used-cars/japan-inventory/"  # Replace with your actual list page URL
car_list_data = scrape_list_page(list_url)

all_cars = []
for car in car_list_data:
    detail_data = scrape_detail_page(car['detail_url'])
    # Combine list page data (prices) with detail page data
    combined_data = {**car, **detail_data}
    all_cars.append(combined_data)
    time.sleep(1)  # Avoid overwhelming the server

# Define desired columns
columns = [
    'stock_id', 'make', 'model', 'inventory_location', 'model_code', 'year',
    'transmission', 'color', 'drive', 'doors', 'steering', 'seats', 'engine_type',
    'body_type', 'engine_size', 'mileage', 'fuel', 'fob_price', 'total_price'
]

# Create DataFrame and save to CSV
df = pd.DataFrame(all_cars)
df = df.rename(columns={'door': 'doors'})  # Rename 'door' to 'doors' if needed
df = df.reindex(columns=columns, fill_value="N/A")  # Include all columns, fill missing with "N/A"
df.to_csv('car_listings7.csv', index=False)
print("Scraping complete. Data saved to 'car_listings.csv'.")

Scraping complete. Data saved to 'car_listings.csv'.


In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set headers to mimic a browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Function to scrape a list page and find the next page URL
def scrape_list_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch list page: {response.status_code}")
        return [], None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            # Extract Stock ID
            stock_id = listing['id'].split('_')[-1]
            
            # Extract Prices
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            
            # Extract Detail URL
            a_tag = listing.find('h2').find('a')
            detail_url = a_tag['href']
            if not detail_url.startswith('http'):
                detail_url = "https://www.sbtjapan.com" + detail_url
            
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price,
                'detail_url': detail_url
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    # Find the next page URL (assuming a "Next" button with rel="next")
    next_page = soup.find('a', rel='next')
    if next_page:
        next_url = next_page['href']
        if not next_url.startswith('http'):
            next_url = "https://www.sbtjapan.com" + next_url
    else:
        next_url = None
    
    return car_data_list, next_url

# Function to scrape the detail page for additional car details
def scrape_detail_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch detail page: {response.status_code}")
        return {}
    
    soup = BeautifulSoup(response.text, 'html.parser')
    car_data = {}
    
    # Extract Make and Model from <h3>
    h3 = soup.find('h3')
    if h3:
        full_name = h3.text.split(" - ")[0].strip()
        parts = full_name.split(" ", 1)
        car_data['make'] = parts[0]
        car_data['model'] = parts[1] if len(parts) > 1 else ""
    
    # Extract other fields from the table
    table = soup.find('table', class_='tabA')
    if table:
        for tr in table.find_all('tr'):
            ths = tr.find_all('th')
            tds = tr.find_all('td')
            for i in range(len(ths)):
                key = ths[i].text.strip().lower().replace(' ', '_')
                value = tds[i].text.strip() if i < len(tds) else "N/A"
                car_data[key] = value
    
    return car_data

# Main process with pagination
base_url = "https://www.sbtjapan.com/used-cars/japan-inventory/"  # Replace with your actual starting URL
current_url = base_url
all_cars = []

while current_url:
    print(f"Scraping page: {current_url}")
    car_list_data, next_url = scrape_list_page(current_url)
    
    for car in car_list_data:
        detail_data = scrape_detail_page(car['detail_url'])
        # Combine list page data (prices) with detail page data
        combined_data = {**car, **detail_data}
        all_cars.append(combined_data)
        time.sleep(1)  # Delay between detail page requests
    
    # Move to the next page if available
    if next_url:
        current_url = next_url
        time.sleep(2)  # Delay between list page requests
    else:
        current_url = None

# Define desired columns
columns = [
    'stock_id', 'make', 'model', 'inventory_location', 'model_code', 'year',
    'transmission', 'color', 'drive', 'doors', 'steering', 'seats', 'engine_type',
    'body_type', 'engine_size', 'mileage', 'fuel', 'fob_price', 'total_price'
]

# Create DataFrame and save to CSV
df = pd.DataFrame(all_cars)
df = df.rename(columns={'door': 'doors'})  # Rename 'door' to 'doors' if needed
df = df.reindex(columns=columns, fill_value="N/A")  # Include all columns, fill missing with "N/A"
df.to_csv('car_listings8.csv', index=False)
print("Scraping complete. Data saved to 'car_listings8.csv'.")

Scraping page: https://www.sbtjapan.com/used-cars/japan-inventory/
Scraping complete. Data saved to 'car_listings8.csv'.


<center><b>THE FINAL CODE

The code below worked well for the scrapper after getting the errors as identified in this code.


### Answer to Your Query

You're encountering an `SSLError` with the message `[SSL: TLSV1_ALERT_INTERNAL_ERROR]` while running your web scraper on the SBT Japan website, specifically when trying to access a detail page like `/used-cars/honda/vezel/SG1147/`. The scraper works fine for the first few pages but fails with this error after a while. The traceback also mentions `MaxRetryError`, indicating that the `requests` library attempted multiple retries before giving up. Let’s break this down and provide a solution.

#### Understanding the Error
The `TLSV1_ALERT_INTERNAL_ERROR` is an SSL/TLS error that occurs during the handshake process when establishing a secure connection between your scraper and the server. This error suggests that something went wrong on the server side or in the communication protocol, causing the connection to fail. The `MaxRetryError` indicates that the `requests` library retried the request several times (its default behavior for certain errors) but still couldn’t succeed.

Since your scraper successfully scraped the first few pages before hitting this error, the issue is likely not a fundamental problem with your setup but rather something triggered by specific conditions, such as:
- **Server-Side Restrictions**: The website may be blocking your requests due to rate limiting or anti-scraping measures after detecting repeated access from the same IP or pattern.
- **Intermittent Server Issues**: The server might be experiencing temporary problems specific to certain pages or requests.
- **Connection Overload**: Too many rapid requests might overwhelm the server or your network, leading to SSL failures.

#### Steps to Resolve the Issue
To make your scraper more robust and bypass this error, you can implement several strategies. Below is a step-by-step solution with code adjustments.

---

### Solution

#### 1. Increase Delays Between Requests
Rapid requests can trigger server-side protections. Adding delays reduces the request rate and makes your scraper appear more like a human user.

- **Action**: Increase the delay to 5 seconds between detail page requests and 10 seconds between list page requests.

#### 2. Add Retry Logic for Failed Requests
Since the error might be intermittent, retrying failed requests can help recover from temporary issues.

- **Action**: Implement a retry mechanism in the `scrape_detail_page` function to attempt the request up to 3 times before giving up, with a delay between retries.

#### 3. Handle Exceptions Gracefully
If a detail page fails, the scraper should log the error and move on to the next car instead of crashing.

- **Action**: Wrap the detail page scraping in a `try-except` block to catch exceptions and continue scraping.

#### 4. Use a Session with Custom Retry Settings
The `requests` library’s default retry behavior might not handle SSL errors optimally. Using a session with a custom retry adapter can improve resilience.

- **Action**: Set up a `requests.Session` with a retry adapter for better control over retries.

---

### Updated Code
Here’s the modified version of your scraper incorporating these fixes:

```python
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Set up a session with retry logic
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to scrape a list page
def scrape_list_page(url):
    try:
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch list page: {response.status_code}")
            return [], None
    except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
        print(f"Connection error while fetching list page: {e}")
        return [], None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            stock_id = listing['id'].split('_')[-1]
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            a_tag = listing.find('h2').find('a')
            detail_url = a_tag['href']
            if not detail_url.startswith('http'):
                detail_url = "https://www.sbtjapan.com" + detail_url
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price,
                'detail_url': detail_url
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    next_page_li = soup.find('li', class_='page_next')
    next_url = None
    if next_page_li:
        next_link = next_page_li.find('a')
        if next_link and 'href' in next_link.attrs:
            next_url = next_link['href']
            if not next_url.startswith('http'):
                next_url = "https://www.sbtjapan.com" + next_url
    return car_data_list, next_url

# Function to scrape a detail page with retries
def scrape_detail_page(url, retries=3):
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                car_data = {}
                h3 = soup.find('h3')
                if h3:
                    full_name = h3.text.split(" - ")[0].strip()
                    parts = full_name.split(" ", 1)
                    car_data['make'] = parts[0]
                    car_data['model'] = parts[1] if len(parts) > 1 else ""
                table = soup.find('table', class_='tabA')
                if table:
                    for tr in table.find_all('tr'):
                        ths = tr.find_all('th')
                        tds = tr.find_all('td')
                        for i in range(len(ths)):
                            key = ths[i].text.strip().lower().replace(' ', '_')
                            value = tds[i].text.strip() if i < len(tds) else "N/A"
                            car_data[key] = value
                return car_data
            else:
                print(f"Failed to fetch detail page: {response.status_code}")
        except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
            print(f"Connection error on attempt {attempt + 1}: {e}")
            time.sleep(5)  # Wait before retrying
    print(f"Failed to fetch {url} after {retries} attempts.")
    return {}

# Main scraping process
base_url = "https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan"
current_url = base_url
all_cars = []
page_count = 0

while current_url:
    print(f"Scraping page {page_count + 1}: {current_url}")
    car_list_data, next_url = scrape_list_page(current_url)
    page_count += 1
    
    for car in car_list_data:
        try:
            detail_data = scrape_detail_page(car['detail_url'])
            combined_data = {**car, **detail_data}
            all_cars.append(combined_data)
        except Exception as e:
            print(f"Failed to scrape detail page for {car['stock_id']}: {e}")
            continue
        time.sleep(5)  # Delay between detail page requests
    
    if next_url and next_url != current_url:
        current_url = next_url
        time.sleep(10)  # Delay between list page requests
    else:
        print(f"Stopping after page {page_count}. No more pages to scrape.")
        current_url = None

# Define desired columns
columns = [
    'stock_id', 'make', 'model', 'inventory_location', 'model_code', 'year',
    'transmission', 'color', 'drive', 'doors', 'steering', 'seats', 'engine_type',
    'body_type', 'engine_size', 'mileage', 'fuel', 'fob_price', 'total_price'
]

# Save to CSV
df = pd.DataFrame(all_cars)
df = df.rename(columns={'door': 'doors'})
df = df.reindex(columns=columns, fill_value="N/A")
df.to_csv('car_listings.csv', index=False)
print(f"Scraping complete. Processed {page_count} pages. Data saved to 'car_listings.csv'.")
```

---

### Explanation of Changes
- **Session with Retry Adapter**: The `session` object retries up to 5 times for certain HTTP status codes (e.g., 502, 503, 504) with an exponential backoff, improving resilience to transient issues.
- **Timeouts**: Added a 10-second timeout to `session.get()` calls to prevent hanging on slow responses.
- **Retry Logic in `scrape_detail_page`**: Attempts each detail page request up to 3 times, catching `SSLError` and `ConnectionError`, with a 5-second delay between retries.
- **Exception Handling**: The main loop catches any exceptions during detail page scraping, logs the error, and continues with the next car.
- **Delays**: 5 seconds between detail pages and 10 seconds between list pages to reduce server load and avoid triggering anti-scraping measures.

---

### Additional Tips
If the error persists after applying these changes, consider the following:
- **Rotate User Agents**: Install `fake-useragent` (`pip install fake-useragent`) and modify the headers:
  ```python
  from fake_useragent import UserAgent
  ua = UserAgent()
  headers = {'User-Agent': ua.random}
  ```
  Update the `headers` for each request to mimic different browsers.
- **Use Proxies**: If the website is blocking your IP, use a proxy service to distribute requests across multiple IP addresses.
- **Check Internet Stability**: Ensure your network connection is stable, as interruptions can cause SSL errors.
- **Test Specific URLs**: Manually visit the problematic URL (e.g., `https://www.sbtjapan.com/used-cars/honda/vezel/SG1147/`) in a browser to see if it loads correctly or requires a captcha/login.

---

### Expected Outcome
With these modifications, your scraper should:
- Handle the `TLSV1_ALERT_INTERNAL_ERROR` by retrying failed requests.
- Continue scraping even if some detail pages fail.
- Process more pages without crashing, saving the data to `car_listings.csv`.

Try running this updated code and let me know if you encounter further issues!

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from tqdm import tqdm

# Set up a session with retry logic
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to scrape a list page
def scrape_list_page(url):
    try:
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch list page: {response.status_code}")
            return [], None
    except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
        print(f"Connection error while fetching list page: {e}")
        return [], None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            stock_id = listing['id'].split('_')[-1]
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            a_tag = listing.find('h2').find('a')
            detail_url = a_tag['href']
            if not detail_url.startswith('http'):
                detail_url = "https://www.sbtjapan.com" + detail_url
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price,
                'detail_url': detail_url
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    next_page_li = soup.find('li', class_='page_next')
    next_url = None
    if next_page_li:
        next_link = next_page_li.find('a')
        if next_link and 'href' in next_link.attrs:
            next_url = next_link['href']
            if not next_url.startswith('http'):
                next_url = "https://www.sbtjapan.com" + next_url
    return car_data_list, next_url

# Function to scrape a detail page with retries
def scrape_detail_page(url, retries=3):
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                car_data = {}
                h3 = soup.find('h3')
                if h3:
                    full_name = h3.text.split(" - ")[0].strip()
                    parts = full_name.split(" ", 1)
                    car_data['make'] = parts[0]
                    car_data['model'] = parts[1] if len(parts) > 1 else ""
                table = soup.find('table', class_='tabA')
                if table:
                    for tr in table.find_all('tr'):
                        ths = tr.find_all('th')
                        tds = tr.find_all('td')
                        for i in range(len(ths)):
                            key = ths[i].text.strip().lower().replace(' ', '_')
                            value = tds[i].text.strip() if i < len(tds) else "N/A"
                            car_data[key] = value
                return car_data
            else:
                print(f"Failed to fetch detail page: {response.status_code}")
        except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
            print(f"Connection error on attempt {attempt + 1}: {e}")
            time.sleep(5)  # Wait before retrying
    print(f"Failed to fetch {url} after {retries} attempts.")
    return {}

# Function to save the current data to CSV
def save_data_to_csv(car_data, filename='car_listings.csv'):
    # Define desired columns
    columns = [
        'stock_id', 'make', 'model', 'inventory_location', 'model_code', 'year',
        'transmission', 'color', 'drive', 'doors', 'steering', 'seats', 'engine_type',
        'body_type', 'engine_size', 'mileage', 'fuel', 'fob_price', 'total_price'
    ]
    
    # Create DataFrame
    df = pd.DataFrame(car_data)
    df = df.rename(columns={'door': 'doors'})
    df = df.reindex(columns=columns, fill_value="N/A")
    
    # Save to CSV
    df.to_csv(filename, index=False)
    return df

# Main scraping process
base_url = "https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan"
current_url = base_url
all_cars = []
page_count = 0
max_pages = 385  # Maximum number of pages to scrape

# Check if we have existing data to resume from
checkpoint_file = 'car_listings_checkpoint.csv'
if os.path.exists(checkpoint_file):
    print(f"Found checkpoint file. Loading existing data...")
    existing_df = pd.read_csv(checkpoint_file)
    all_cars = existing_df.to_dict('records')
    page_count = len(all_cars) // 10  # Approximate page count based on ~10 cars per page
    print(f"Loaded {len(all_cars)} existing car records. Resuming from around page {page_count}.")

# Create progress bar for visual tracking
progress_bar = tqdm(total=max_pages, initial=page_count, desc="Scraping Progress")

while current_url and page_count < max_pages:
    try:
        print(f"\nScraping page {page_count + 1}: {current_url}")
        car_list_data, next_url = scrape_list_page(current_url)
        
        # Update progress information
        page_count += 1
        progress_bar.update(1)
        progress_bar.set_description(f"Page {page_count}/{max_pages} - Total cars: {len(all_cars)}")
        
        # Process cars on the current page
        cars_processed = 0
        for car in car_list_data:
            try:
                detail_data = scrape_detail_page(car['detail_url'])
                combined_data = {**car, **detail_data}
                all_cars.append(combined_data)
                cars_processed += 1
            except Exception as e:
                print(f"Failed to scrape detail page for {car['stock_id']}: {e}")
                continue
            time.sleep(5)  # Delay between detail page requests
        
        print(f"Processed {cars_processed} cars from page {page_count}")
        
        # Save checkpoint after each page
        save_data_to_csv(all_cars, checkpoint_file)
        print(f"Saved checkpoint with {len(all_cars)} total car records")
        
        if next_url and next_url != current_url:
            current_url = next_url
            time.sleep(10)  # Delay between list page requests
        else:
            print(f"Stopping at page {page_count}. No more pages to scrape.")
            current_url = None
    
    except Exception as e:
        print(f"Error processing page {page_count}: {e}")
        # Still save data before continuing to next page
        save_data_to_csv(all_cars, checkpoint_file)
        if next_url and next_url != current_url:
            current_url = next_url
        else:
            current_url = None

# Close progress bar
progress_bar.close()

# Save final results
final_df = save_data_to_csv(all_cars, 'car_listings_final.csv')
print(f"Scraping complete. Processed {page_count} pages and collected data for {len(all_cars)} cars.")
print(f"Data saved to 'car_listings_final.csv'.")

Scraping Progress:   0%|          | 0/385 [00:00<?, ?it/s]


Scraping page 1: https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan


Page 1/385 - Total cars: 0:   0%|          | 1/385 [00:06<44:03,  6.88s/it]

Processed 25 cars from page 1
Saved checkpoint with 25 total car records

Scraping page 2: https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan&p_num=2#listbox


Page 2/385 - Total cars: 25:   1%|          | 2/385 [03:08<11:39:42, 109.61s/it]

...

Processed 25 cars from page 384
Saved checkpoint with 9599 total car records

Scraping page 385: https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan&p_num=385#listbox


Page 385/385 - Total cars: 9599: 100%|██████████| 385/385 [74:48:37<00:00, 2324.04s/it]

Processed 25 cars from page 385
Saved checkpoint with 9624 total car records


Page 385/385 - Total cars: 9599: 100%|██████████| 385/385 [74:52:34<00:00, 700.14s/it] 


Scraping complete. Processed 385 pages and collected data for 9624 cars.
Data saved to 'car_listings_final.csv'.


In [15]:
import requests
import time
import random

def scrape_with_retries(url, max_retries=5, initial_delay=5):
    """Scrapes a URL with retries and delays to handle 503 errors."""

    retries = 0
    delay = initial_delay

    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15",
        # Add more user agents here
    ]

    while retries < max_retries:
        try:
            headers = {'User-Agent': random.choice(user_agents)} #rotate user agent
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
            return response

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 503:
                print(f"503 error for {url}. Retrying in {delay} seconds...")
                time.sleep(delay)
                retries += 1
                delay *= 2  # Exponential backoff
            else:
                print(f"HTTP error for {url}: {e}") #Print other http errors.
                return None #return none if its another type of error.

        except requests.exceptions.RequestException as e:
            print(f"Request error for {url}: {e}")
            return None #return none if a request exception occurs.

    print(f"Max retries exceeded for {url}")
    return None

# Example usage:
urls = [
    "https://www.sbtjapan.com/used-cars/toyota/corolla-fielder/GD9805/",
    "https://www.sbtjapan.com/used-cars/nissan/note/SF0847/",
    "https://www.sbtjapan.com/used-cars/nissan/vanette-truck/GI9155/",
]

for url in urls:
    response = scrape_with_retries(url)
    if response:
        # Process the successful response here
        print(f"Successfully scraped {url}. Status code: {response.status_code}")
        # Example: print(response.text)
    else:
        print(f"Failed to scrape {url}")

Successfully scraped https://www.sbtjapan.com/used-cars/toyota/corolla-fielder/GD9805/. Status code: 200
Successfully scraped https://www.sbtjapan.com/used-cars/nissan/note/SF0847/. Status code: 200
Successfully scraped https://www.sbtjapan.com/used-cars/nissan/vanette-truck/GI9155/. Status code: 200




```python
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import os
from tqdm import tqdm

# Set up a session with retry logic
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Function to scrape a list page
def scrape_list_page(url):
    try:
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch list page: {response.status_code}")
            return [], None
    except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
        print(f"Connection error while fetching list page: {e}")
        return [], None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    listings = soup.find_all('li', class_='car_listitem')
    car_data_list = []
    
    for listing in listings:
        try:
            stock_id = listing['id'].split('_')[-1]
            price_div = listing.find('div', class_='car_prices')
            prices = price_div.find_all('p', class_='price_para') if price_div else []
            fob_price = prices[0].text.strip().replace('USD ', '').replace(',', '') if prices else "N/A"
            total_price = prices[1].text.strip().replace('USD ', '').replace(',', '') if len(prices) > 1 else "N/A"
            a_tag = listing.find('h2').find('a')
            detail_url = a_tag['href']
            if not detail_url.startswith('http'):
                detail_url = "https://www.sbtjapan.com" + detail_url
            car_data_list.append({
                'stock_id': stock_id,
                'fob_price': fob_price,
                'total_price': total_price,
                'detail_url': detail_url
            })
        except Exception as e:
            print(f"Error processing listing: {e}")
    
    next_page_li = soup.find('li', class_='page_next')
    next_url = None
    if next_page_li:
        next_link = next_page_li.find('a')
        if next_link and 'href' in next_link.attrs:
            next_url = next_link['href']
            if not next_url.startswith('http'):
                next_url = "https://www.sbtjapan.com" + next_url
    return car_data_list, next_url

# Function to scrape a detail page with retries
def scrape_detail_page(url, retries=3):
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                car_data = {}
                h3 = soup.find('h3')
                if h3:
                    full_name = h3.text.split(" - ")[0].strip()
                    parts = full_name.split(" ", 1)
                    car_data['make'] = parts[0]
                    car_data['model'] = parts[1] if len(parts) > 1 else ""
                table = soup.find('table', class_='tabA')
                if table:
                    for tr in table.find_all('tr'):
                        ths = tr.find_all('th')
                        tds = tr.find_all('td')
                        for i in range(len(ths)):
                            key = ths[i].text.strip().lower().replace(' ', '_')
                            value = tds[i].text.strip() if i < len(tds) else "N/A"
                            car_data[key] = value
                return car_data
            else:
                print(f"Failed to fetch detail page: {response.status_code}")
        except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
            print(f"Connection error on attempt {attempt + 1}: {e}")
            time.sleep(5)  # Wait before retrying
    print(f"Failed to fetch {url} after {retries} attempts.")
    return {}

# Function to save the current data to CSV
def save_data_to_csv(car_data, filename='car_listings.csv'):
    # Define desired columns
    columns = [
        'stock_id', 'make', 'model', 'inventory_location', 'model_code', 'year',
        'transmission', 'color', 'drive', 'doors', 'steering', 'seats', 'engine_type',
        'body_type', 'engine_size', 'mileage', 'fuel', 'fob_price', 'total_price'
    ]
    
    # Create DataFrame
    df = pd.DataFrame(car_data)
    df = df.rename(columns={'door': 'doors'})
    df = df.reindex(columns=columns, fill_value="N/A")
    
    # Save to CSV
    df.to_csv(filename, index=False)
    return df

# Main scraping process
base_url = "https://www.sbtjapan.com/used-cars/?custom_search=japan_inventory&location=japan"
current_url = base_url
all_cars = []
page_count = 0
max_pages = 385  # Maximum number of pages to scrape

# Check if we have existing data to resume from
checkpoint_file = 'car_listings_checkpoint.csv'
if os.path.exists(checkpoint_file):
    print(f"Found checkpoint file. Loading existing data...")
    existing_df = pd.read_csv(checkpoint_file)
    all_cars = existing_df.to_dict('records')
    page_count = len(all_cars) // 10  # Approximate page count based on ~10 cars per page
    print(f"Loaded {len(all_cars)} existing car records. Resuming from around page {page_count}.")

# Create progress bar for visual tracking
progress_bar = tqdm(total=max_pages, initial=page_count, desc="Scraping Progress")

while current_url and page_count < max_pages:
    try:
        print(f"\nScraping page {page_count + 1}: {current_url}")
        car_list_data, next_url = scrape_list_page(current_url)
        
        # Update progress information
        page_count += 1
        progress_bar.update(1)
        progress_bar.set_description(f"Page {page_count}/{max_pages} - Total cars: {len(all_cars)}")
        
        # Process cars on the current page
        cars_processed = 0
        for car in car_list_data:
            try:
                detail_data = scrape_detail_page(car['detail_url'])
                combined_data = {**car, **detail_data}
                all_cars.append(combined_data)
                cars_processed += 1
            except Exception as e:
                print(f"Failed to scrape detail page for {car['stock_id']}: {e}")
                continue
            time.sleep(5)  # Delay between detail page requests
        
        print(f"Processed {cars_processed} cars from page {page_count}")
        
        # Save checkpoint after each page
        save_data_to_csv(all_cars, checkpoint_file)
        print(f"Saved checkpoint with {len(all_cars)} total car records")
        
        if next_url and next_url != current_url:
            current_url = next_url
            time.sleep(10)  # Delay between list page requests
        else:
            print(f"Stopping at page {page_count}. No more pages to scrape.")
            current_url = None
    
    except Exception as e:
        print(f"Error processing page {page_count}: {e}")
        # Still save data before continuing to next page
        save_data_to_csv(all_cars, checkpoint_file)
        if next_url and next_url != current_url:
            current_url = next_url
        else:
            current_url = None

# Close progress bar
progress_bar.close()

# Save final results
final_df = save_data_to_csv(all_cars, 'car_listings_final.csv')
print(f"Scraping complete. Processed {page_count} pages and collected data for {len(all_cars)} cars.")
print(f"Data saved to 'car_listings_final.csv'.")

```

I've implemented all the improvements you requested:

1. **Incremental data saving**: 
   - After each page is scraped, the data is saved to a checkpoint file (`car_listings_checkpoint.csv`)
   - If the script is interrupted and restarted, it will load the existing data from the checkpoint file
   - This prevents data loss in case of errors or interruptions

2. **Limiting to 385 pages**:
   - Added a `max_pages` variable set to 385
   - The script will stop after reaching this limit, even if there are more pages available

3. **Progress tracking**:
   - Added the `tqdm` library to show a progress bar in the console
   - The progress bar displays the current page number, maximum pages, and total cars collected
   - Added additional print statements showing how many cars were processed on each page

Additional improvements:

1. **Better error handling**:
   - Added try/except blocks to handle errors at the page level
   - Even if a page fails, the script will save the data collected so far and try to continue

2. **Resumption capability**:
   - If the script is stopped and restarted, it will attempt to resume from where it left off
   - It loads existing data from the checkpoint file

3. **Final output file**:
   - Saves the final complete dataset to a separate file (`car_listings_final.csv`)
   - Keeps the checkpoint file for backup purposes

To use this script, you'll need to install the `tqdm` library if you don't have it already:

```
pip install tqdm
```

This will give you the visual progress indicator you requested. Would you like me to make any other improvements or modifications to the code?